In [ ]:
!pip install networkx

In [ ]:
import networkx as nx

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


#path = "/content/drive/MyDrive/MyDatasets/network.dat"
#path = "/content/drive/MyDrive/MyDatasets/network.dat"
#path = "/content/drive/MyDrive/MyDatasets/2.Riskmap.csv"
#path = "/content/drive/MyDrive/MyDatasets/10.power.txt"
#path = "/content/drive/MyDrive/1.KARATE.csv"
path = "/content/drive/MyDrive/12.sawmill.csv"
#3.polbooks.txt"11.Strike.csv

Mounted at /content/drive


In [ ]:
# PATH = "/content/drive/MyDrive/3.polbooks.txt"
PATH = "/content/drive/MyDrive/12.sawmill.csv"
GT_COMM_PATH = "/content/drive/MyDrive/15rw_comm_t03.txt"


In [2]:
#NEW_HPLMC

import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict
from math import sqrt
from sklearn.metrics import normalized_mutual_info_score
from networkx.algorithms.community.quality import modularity

# -------------------------------------------------
# Load Karate graph
# -------------------------------------------------
# G = nx.karate_club_graph()
# print("Graph loaded:", G.number_of_nodes(), "nodes,", G.number_of_edges(), "edges")

# # Ground truth
# gt_labels = {}
# for n, d in G.nodes(data=True):
#     gt_labels[n] = 0 if d["club"] == "Mr. Hi" else 1

# -------------------------------------------------
# Load Karate graph from CSV
# -------------------------------------------------
EDGE_PATH = "boston.csv" # 4.football.csv,, 2.Riskmap.csv, 3.polbooks.txt, 11.Strike.csv, 12. sawmill.csv
#GT_COMM_PATH = "/content/drive/MyDrive/15rw_comm_t05.txt"

def load_graph_from_csv(path):
    G = nx.Graph()
    with open(path, "r") as f:
        for line in f:
            if not line.strip():
                continue
            parts = line.replace(",", " ").split()
            if len(parts) < 2:
                continue
            try:
                u = int(parts[0])
                v = int(parts[1])
            except:
                continue
            if u != v:
                G.add_edge(u, v)
    return G

G = load_graph_from_csv(EDGE_PATH)
print("Graph loaded:", G.number_of_nodes(), "nodes,", G.number_of_edges(), "edges")


# -------------------------------------------------
# Load ground-truth communities
# Format:
# each line = one community
# -------------------------------------------------
def load_ground_truth(path):
    gt = {}
    with open(path, "r") as f:
        for label, line in enumerate(f):
            nodes = line.strip().split()
            for n in nodes:
                gt[int(n)] = label
    return gt

#gt_labels = load_ground_truth(GT_COMM_PATH)
#print("Ground truth labels loaded:", len(gt_labels))


# -------------------------------------------------
# Sanity check
# -------------------------------------------------
# missing = set(G.nodes()) - set(gt_labels.keys())
# if missing:
#     print("WARNING: Missing GT labels for nodes:", missing)
# #else:
#     #print("Ground truth covers all nodes.")


# Similarity measures
# -------------------------------------------------
nbrs = {n: set(G.neighbors(n)) for n in G.nodes()}
deg = dict(G.degree())

def sim_adamic_adar(v, u):
    return sum(1 / np.log(deg[z]) for z in nbrs[v] & nbrs[u] if deg[z] > 1)

def sim_resource_allocation(v, u):
    return sum(1 / deg[z] for z in nbrs[v] & nbrs[u] if deg[z] > 0)

def sim_salton(v, u):
    return len(nbrs[v] & nbrs[u]) / sqrt(max(1, deg[v] * deg[u]))

def sim_preferential_attachment(v, u):
    return deg[v] * deg[u]

SIM_FUNCS = {
    "adamic_adar": sim_adamic_adar,
    "preferential_attachment": sim_preferential_attachment,
    "resource_allocation": sim_resource_allocation,
    "salton": sim_salton
}

# -------------------------------------------------
# Reweighted graph construction
# w(u,v) = adjacency + α·triangles + β·3-paths
# -------------------------------------------------
# def compute_reweighted_graph(G, alpha=1.0, beta=0.5):
#     G_rw = nx.Graph()
#     G_rw.add_nodes_from(G.nodes())

#     # --- triangle counts ---
#     tri_weight = defaultdict(int)
#     for v in G.nodes():
#         neigh = list(G.neighbors(v))
#         for i in range(len(neigh)):
#             for j in range(i + 1, len(neigh)):
#                 u, w = neigh[i], neigh[j]
#                 if G.has_edge(u, w):
#                     tri_weight[(u, v)] += 1
#                     tri_weight[(v, w)] += 1
#                     tri_weight[(u, w)] += 1

#     # --- 3-path counts ---
#     path_weight = defaultdict(int)
#     for x in G.nodes():
#         nbrs_x = list(G.neighbors(x))
#         for i in range(len(nbrs_x)):
#             for j in range(i + 1, len(nbrs_x)):
#                 u, v = nbrs_x[i], nbrs_x[j]
#                 path_weight[(u, v)] += 1
#                 path_weight[(v, u)] += 1

#     # --- build weighted edges ---
#     for u, v in G.edges():
#         w = (
#             1.0
#             + alpha * tri_weight.get((u, v), 0)
#             + beta * path_weight.get((u, v), 0)
#         )
#         G_rw.add_edge(u, v, weight=w)

#     return G_rw
# def compute_reweighted_graph(G, alpha=1.0, beta=0.0):
#     G_rw = nx.Graph()
#     G_rw.add_nodes_from(G.nodes())

#     n = G.number_of_nodes()

    # --- all-pairs shortest paths ---
    # dist = dict(nx.all_pairs_shortest_path_length(G))

    # # --- edge closeness centrality ---
    # edge_closeness = {}

    # for u, v in G.edges():
    #     total_dist = 0
    #     for x in G.nodes():
    #         total_dist += min(dist[u][x], dist[v][x])

    #     # avoid division by zero (theoretically impossible in connected graphs)
    #     edge_closeness[(u, v)] = n / total_dist if total_dist > 0 else 0.0
    #     edge_closeness[(v, u)] = edge_closeness[(u, v)]

    # # --- build weighted edges ---
    # for u, v in G.edges():
    #     w = 1.0 + alpha * edge_closeness[(u, v)]
    #     G_rw.add_edge(u, v, weight=w)

    # return G_rw


# def compute_reweighted_graph(G, alpha=1.0, beta=0.0):
#     G_rw = nx.Graph()
#     G_rw.add_nodes_from(G.nodes())

#     n = G.number_of_nodes()
#     deg = dict(G.degree())
#     nbrs = {u: set(G.neighbors(u)) for u in G.nodes()}

#     # --- shortest paths ---
#     dist = dict(nx.all_pairs_shortest_path_length(G))

#     for u, v in G.edges():
#         # ----- 1. softened edge closeness -----
#         total_dist = 0
#         for x in G.nodes():
#             total_dist += min(dist[u][x], dist[v][x])
#         Ce = 1.0 / total_dist

#         # ----- 2. triangle density -----
#         tri = len(nbrs[u] & nbrs[v])
#         T = (tri + 1) / np.sqrt(max(1, deg[u] * deg[v]))

#         # ----- 3. hub regularization -----
#         R = 1.0 / np.log(2 + deg[u] + deg[v])

#         # ----- final weight -----
#         w = 1.0 + alpha * Ce * T * R

#         G_rw.add_edge(u, v, weight=w)

#     return G_rw
# def compute_reweighted_graph(G, alpha=1.0, beta=0.0):
#     G_rw = nx.Graph()
#     G_rw.add_nodes_from(G.nodes())

#     deg = dict(G.degree())
#     nbrs = {u: set(G.neighbors(u)) for u in G.nodes()}

#     # all-pairs shortest paths (small graphs OK)
#     dist = dict(nx.all_pairs_shortest_path_length(G))

#     for u, v in G.edges():
#         # ----- local cohesion (dominant) -----
#         shared = len(nbrs[u] & nbrs[v])
#         T = (shared + 1) / max(1, min(deg[u], deg[v]))

#         # ----- gated global term -----
#         total_dist = 0
#         for x in G.nodes():
#             total_dist += min(dist[u][x], dist[v][x])
#         G_term = 1.0 / (1.0 + total_dist)

#         # ----- final weight -----
#         w = 1.0 + alpha * T * G_term
#         G_rw.add_edge(u, v, weight=w)

#     return G_rw

# def compute_reweighted_graph(G, alpha=1.0, beta=0.5):
#     G_rw = nx.Graph()
#     G_rw.add_nodes_from(G.nodes())

#     deg = dict(G.degree())
#     nbrs = {u: set(G.neighbors(u)) for u in G.nodes()}

#     # --- triangle counts (edge-based) ---
#     tri_weight = defaultdict(int)
#     for v in G.nodes():
#         neigh = list(G.neighbors(v))
#         for i in range(len(neigh)):
#             for j in range(i + 1, len(neigh)):
#                 u, w = neigh[i], neigh[j]
#                 if G.has_edge(u, w):
#                     tri_weight[(u, v)] += 1
#                     tri_weight[(v, w)] += 1
#                     tri_weight[(u, w)] += 1

#     # --- 3-path counts ---
#     path_weight = defaultdict(int)
#     for x in G.nodes():
#         nbrs_x = list(G.neighbors(x))
#         for i in range(len(nbrs_x)):
#             for j in range(i + 1, len(nbrs_x)):
#                 u, v = nbrs_x[i], nbrs_x[j]
#                 path_weight[(u, v)] += 1
#                 path_weight[(v, u)] += 1

#     # --- build weighted edges ---
#     for u, v in G.edges():
#         T = (tri_weight.get((u, v), 0) + 1) / np.sqrt(max(1, deg[u] * deg[v]))
#         P = path_weight.get((u, v), 0) / max(1, min(deg[u], deg[v]))

#         w = 1.0 + alpha * T + beta * P
#         G_rw.add_edge(u, v, weight=w)

#     return G_rw


# def compute_reweighted_graph(G, alpha=1.0, beta=0.5):
#     G_rw = nx.Graph()
#     G_rw.add_nodes_from(G.nodes())

#     deg = dict(G.degree())
#     nbrs = {u: set(G.neighbors(u)) for u in G.nodes()}

#     # --- triangle counts ---
#     tri_weight = defaultdict(int)
#     for v in G.nodes():
#         neigh = list(G.neighbors(v))
#         for i in range(len(neigh)):
#             for j in range(i + 1, len(neigh)):
#                 u, w = neigh[i], neigh[j]
#                 if G.has_edge(u, w):
#                     tri_weight[(u, v)] += 1
#                     tri_weight[(v, w)] += 1
#                     tri_weight[(u, w)] += 1

#     # --- 3-path counts ---
#     path_weight = defaultdict(int)
#     for x in G.nodes():
#         nbrs_x = list(G.neighbors(x))
#         for i in range(len(nbrs_x)):
#             for j in range(i + 1, len(nbrs_x)):
#                 u, v = nbrs_x[i], nbrs_x[j]
#                 path_weight[(u, v)] += 1
#                 path_weight[(v, u)] += 1

#     # --- build weighted edges ---
#     for u, v in G.edges():
#         T_raw = tri_weight.get((u, v), 0)
#         P_raw = path_weight.get((u, v), 0)

#         # normalized triangle term
#         T = (T_raw + 1) / np.sqrt(max(1, deg[u] * deg[v]))

#         # gated 3-path term
#         P = P_raw / max(1, min(deg[u], deg[v]))

#         # edge embeddedness (conductance control)
#         emb = T_raw / (min(deg[u] - 1, deg[v] - 1) + 1)

#         w = 1.0 + (alpha * T + beta * P) * emb
#         G_rw.add_edge(u, v, weight=w)

#     return G_rw

def compute_reweighted_graph(G):
    G_rw = nx.Graph()
    G_rw.add_nodes_from(G.nodes())

    deg = dict(G.degree())
    nbrs = {u: set(G.neighbors(u)) for u in G.nodes()}

    for u, v in G.edges():
        common = nbrs[u] & nbrs[v]
        c = len(common)

        # --- Triangle probability (Jaccard) ---
        union_size = len(nbrs[u] | nbrs[v])
        T = c / union_size if union_size > 0 else 0.0

        # --- Two-step proximity (local closeness surrogate) ---
        if c > 0:
            P = sum(1 / deg[x] for x in common) / c
        else:
            P = 0.0

        # --- Embeddedness damping ---
        E = c / min(deg[u], deg[v]) if min(deg[u], deg[v]) > 0 else 0.0

        w = 1.0 + (T + P) * E
        G_rw.add_edge(u, v, weight=w)

    return G_rw


# def compute_reweighted_graph(G, alpha=1.0, beta=0.5):
#     G_rw = nx.Graph()
#     G_rw.add_nodes_from(G.nodes())

#     deg = dict(G.degree())
#     nbrs = {u: set(G.neighbors(u)) for u in G.nodes()}

#     # --- triangle counts ---
#     tri_weight = defaultdict(int)
#     for v in G.nodes():
#         neigh = list(G.neighbors(v))
#         for i in range(len(neigh)):
#             for j in range(i + 1, len(neigh)):
#                 u, w = neigh[i], neigh[j]
#                 if G.has_edge(u, w):
#                     tri_weight[(u, v)] += 1
#                     tri_weight[(v, w)] += 1
#                     tri_weight[(u, w)] += 1

#     # --- 3-path counts ---
#     path_weight = defaultdict(int)
#     for x in G.nodes():
#         nbrs_x = list(G.neighbors(x))
#         for i in range(len(nbrs_x)):
#             for j in range(i + 1, len(nbrs_x)):
#                 u, v = nbrs_x[i], nbrs_x[j]
#                 path_weight[(u, v)] += 1
#                 path_weight[(v, u)] += 1

#     # --- build weighted edges ---
#     for u, v in G.edges():
#         T_raw = tri_weight.get((u, v), 0)
#         P_raw = path_weight.get((u, v), 0)

#         # triangle term (always active)
#         T = (T_raw + 1) / np.sqrt(max(1, deg[u] * deg[v]))

#         # gated 3-path term (ONLY if triangles exist)
#         gate = T_raw / (T_raw + 1)
#         P = gate * (P_raw / max(1, min(deg[u], deg[v])))

#         w = 1.0 + alpha * T + beta * P
#         G_rw.add_edge(u, v, weight=w)

#     return G_rw



# -------------------------------------------------
# SAMOOH (Label Propagation on Reweighted Graph)
# -------------------------------------------------
def h_lpmc(G_rw, sim_func, max_iter=100):
    deg = dict(G_rw.degree(weight="weight"))
    order = sorted(G_rw.nodes(), key=lambda x: deg[x], reverse=True)

    labels = {v: v for v in G_rw.nodes()}

    for _ in range(max_iter):
        changed = False
        for v in order:
            best_score = -1e18
            best_nbr = None

            for u in G_rw.neighbors(v):
                w = G_rw[v][u]["weight"]
                score = w * sim_func(v, u)
                #score = w

                if score > best_score:
                    best_score = score
                    best_nbr = u

            if best_nbr is not None and labels[v] != labels[best_nbr]:
                labels[v] = labels[best_nbr]
                changed = True

        if not changed:
            break

    return labels

# -------------------------------------------------
# Conductance (UNCHANGED)
# -------------------------------------------------
def conductance_of_partition(G, partition):
    degs = dict(G.degree())
    conds = []
    for S in partition:
        if len(S) == 0 or len(S) == G.number_of_nodes():
            continue
        cut = 0
        volS = 0
        for u in S:
            volS += degs[u]
            for v in G.neighbors(u):
                if v not in S:
                    cut += 1
        volSc = sum(degs[v] for v in G.nodes() if v not in S)
        denom = min(volS, volSc)
        if denom > 0:
            conds.append(cut / denom)
    return float(np.mean(conds)) if conds else np.nan

# -------------------------------------------------
# Utilities
# -------------------------------------------------
def labels_to_communities(labels):
    comms = defaultdict(list)
    for n, l in labels.items():
        comms[l].append(n)
    return list(comms.values())

def plot_communities(G, labels, title):
    pos = nx.spring_layout(G, seed=42)
    comms = labels_to_communities(labels)

    plt.figure(figsize=(7, 6))
    for i, c in enumerate(comms):
        nx.draw_networkx_nodes(G, pos, nodelist=c, node_size=300, label=f"C{i}")
    nx.draw_networkx_edges(G, pos, alpha=0.4)
    nx.draw_networkx_labels(G, pos, font_size=9)
    plt.title(title)
    plt.axis("off")
    plt.legend()
    plt.show()

# -------------------------------------------------
# Run algorithm
# -------------------------------------------------
G_rw = compute_reweighted_graph(G)
labels = h_lpmc(G_rw, sim_adamic_adar)

# -------------------------------------------------
# Evaluation
# -------------------------------------------------
pred = np.array([labels[n] for n in sorted(G.nodes())])
#gt = np.array([gt_labels[n] for n in sorted(G.nodes())])

Q = modularity(G, labels_to_communities(labels))
# NMI = normalized_mutual_info_score(gt, pred)

print(f"Modularity: {Q:.4f}")
# print(f"NMI:        {NMI:.4f}")

# -------------------------------------------------
# Visualization
# -------------------------------------------------
#plot_communities(G, gt_labels, "Ground Truth Communities (Karate)")
plot_communities(G, labels, "Detected Communities (SAMOOH)")
# -------------------------------------------------
# Display communities as list of lists
# -------------------------------------------------
communities = labels_to_communities(labels)


# Run experiments
# -------------------------------------------------
G_rw = compute_reweighted_graph(G)
records = []

for sim_name, sim_func in SIM_FUNCS.items():
    labels = h_lpmc(G_rw, sim_func)
    partition = labels_to_communities(labels)
    modularity_val = modularity(G, partition)
    pred = np.array([labels[n] for n in sorted(G.nodes())])
    #gt = np.array([gt_labels[n] for n in sorted(G.nodes())])
    #nmi_val = normalized_mutual_info_score(gt, pred)
    cond_val = conductance_of_partition(G, partition)

    records.append({
        "similarity": sim_name,
        "modularity_mean": modularity_val,
        # "nmi_mean": nmi_val,
        "conductance_mean": cond_val
    })

# -------------------------------------------------
# Display summary
# -------------------------------------------------
df = pd.DataFrame(records)
#df = df[["similarity", "modularity_mean", "nmi_mean", "conductance_mean"]]
df = df[["similarity", "modularity_mean", "conductance_mean"]]
print("\nSummary Metrics for All Similarity Functions:\n")
print(df.to_string(index=False))
# Display communities as list of lists

# sort for readability
communities = [sorted(c) for c in communities]
communities = sorted(communities, key=len, reverse=True)

print("\nDetected Communities (list of lists):")
for i, c in enumerate(communities):
    print(f"Community {i}: {c}")


Graph loaded: 333045 nodes, 245861 edges
Modularity: 1.0000


KeyboardInterrupt: 